In [ ]:
import os
import numpy as np
import laspy
# import pdal
from time import time
from tqdm import tqdm

In [ ]:
# precise the sample
src_sample = r"D:\PDM_repo\Github\PDM\data\full_dataset\selection\clusters_4\cluster_2\color_grp_full_tile_586.laz"
print(os.path.dirname(src_sample))

In [ ]:
# remove hanging points
laz_in = laspy.read(src_sample)
points = np.array(laz_in.xyz)
voxel_size = 2
voxel_indices = np.floor(points / voxel_size).astype(np.int32)
# print(voxel_indices)
print(voxel_indices.shape)
# points = [
#     [0,1,2],
#     [3,2,2],
#     [-5,10,45]
# ]
min = np.min(points, axis=0)
max = np.max(points, axis=0)
print(min)
print(max)
# min = [1, 1, 2]
# max = [7, 8, 35]
# voxel_size = 4
voxel_indices = []
voxel_indices.append(np.arange(min[0], max[0] + voxel_size, voxel_size))
voxel_indices.append(np.arange(min[1], max[1] + voxel_size, voxel_size))
voxel_indices.append(np.arange(min[2], max[2] + voxel_size, voxel_size))
# print(voxel_x)
# print(voxel_y)
# print(voxel_z)
# print(min[0])
# print(max[0])

a = 0
time_start = time()
container = {x:{y:{z:[] for z in range(len(voxel_indices[2]))} for y in range(len(voxel_indices[1]))} for x in range(len(voxel_indices[0]))}
points_pos_in_container = []
print(container)
for _, point_id in tqdm(enumerate(range(points.shape[0])), total = points.shape[0]):
    full_pos = [0,0,0]
    for ax in range(3):
        for pos in range(len(voxel_indices[ax])):
            if points[point_id, ax] > voxel_indices[ax][pos] and points[point_id, ax] < voxel_indices[ax][pos+1]:
                full_pos[ax] = pos
                break
    # container[0][1][2] = 3
    # print(full_pos[0])
    # print(full_pos[1])
    # print(full_pos[2])
    container[full_pos[0]][full_pos[1]][full_pos[2]].append(points[point_id])
    points_pos_in_container.append(full_pos)
#     for point in range(pos.shape[0]):
#         a += 1
print("Num of ops: ", a)
time_end = time() -time_start
print("Time to do it (s): ", time_end)

In [ ]:
print(len(voxel_indices[0]))
print(len(voxel_indices[1]))
print(len(voxel_indices[2]))
print('---')    
for lst in container[0][0].values():
    print(len(lst))

In [ ]:
THRESHOLD = 5
isolated_points = []
for _, point_id in tqdm(enumerate(range(points.shape[0])), total = points.shape[0]):
    num_neighboors = 0
    pos = points_pos_in_container[point_id]
    for dx in range(-1, 2):
        for dy in range(-1, 2):
            for dz in range(-1, 2):
                x = np.max([pos[0] + dx, 0])
                y = np.max([pos[1] + dy, 0])
                z = np.max([pos[2] + dz, 0])
                num_neighboors += len(container[x][y][z])
    if num_neighboors < THRESHOLD:
        isolated_points.append(point_id)

print(len(isolated_points))


In [ ]:
#save laz in new place
new_file_src = os.path.basename(src_sample).split('.laz')[0] + f'voxel_size_{voxel_size}_isolated_th_{THRESHOLD}.laz'
new_laz_src = os.path.join(os.path.dirname(src_sample), new_file_src)
laz_in.write(new_laz_src)

In [ ]:
if 'isolated' not in list(laz_in.point_format.dimension_names):
    laz_in.add_extra_dim(
        laspy.ExtraBytesParams(
            name='isolated',
            type="f4",
            description='Isolated points',
            ),
        )
laz_in.isolated = np.zeros(len(laz_in), dtype="f4")
for iso_id in isolated_points:
    laz_in.isolated[iso_id] = 1

In [ ]:

# create mask
src_sample = r"D:\PDM_repo\Github\PDM\data\full_dataset\selection\clusters_4\cluster_2\color_grp_full_tile_586.laz"
new_file_src = os.path.basename(src_sample).split('.laz')[0] + 'voxel_size_2_isolated_th_5.laz'
new_laz_src = os.path.join(os.path.dirname(src_sample), new_file_src)
laz_in = laspy.read(new_laz_src)
mask_isolated = laz_in.isolated == 0
print(len(mask_isolated))
print(np.sum(mask_isolated))
print("Num of isolated points: ", int(len(mask_isolated) - np.sum(mask_isolated)))

In [ ]:
# remove points based on mask
laz_in.points = laz_in.points[mask_isolated]
# for dim in laz_in.point_format.dimension_names:
#     laz_in.dim =
print(len(laz_in))
laz_in.write(new_laz_src.split('.laz')[0] + "_removed.laz")